<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable </span> </p>

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

# BenchMarks

In [ ]:
var pathToBm = "./Data/IfrsVariableBenchmarks/";

In [ ]:
public record BenchmarkMetadata(string FileName, string ReportingNode, int Year, int Month){} 

In [ ]:
var bmFiles = new BenchmarkMetadata[]{
    //2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_A.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_AA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_APA.csv" , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_BE.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_BEPA.csv", "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_C.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_DA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_F.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_OA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_RA.csv"  , "CH", 2020, 12),
    //2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_A.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_AA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_AAPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_APA.csv"  , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_BE.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_BEPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_C.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_DA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_F.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_L.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_LR.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_OA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_OAPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3_RA.csv"   , "CH", 2021, 3),
};

# Functions

In [ ]:
public static NumberStyles AllowedNumberStyles = NumberStyles.Float | NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite | NumberStyles.AllowLeadingSign | NumberStyles.AllowTrailingSign | NumberStyles.AllowDecimalPoint;

In [ ]:
public IfrsVariable CreateVariable (IDataRow row, Guid partition)
{
    if (!double.TryParse(row[nameof(IfrsVariable.Value)]?.ToString(), AllowedNumberStyles, CultureInfo.InvariantCulture, out var doubleValue))
        throw new Exception("Value cannot be parsed.");
   
    return new IfrsVariable(){AocType = row.Field<string>(nameof(IfrsVariable.AocType)),
                              Partition = partition,
                              Novelty = row.Field<string>(nameof(IfrsVariable.Novelty)),
                              DataNode = row.Field<string>(nameof(IfrsVariable.DataNode)),
                              EstimateType = row.Field<string>(nameof(IfrsVariable.EstimateType)),
                              AmountType = row.Field<string>(nameof(IfrsVariable.AmountType)),
                              AccidentYear = Int32.TryParse((row.Field<string>(nameof(IfrsVariable.AccidentYear))), out var accidentYear) ? accidentYear : (int?)null,
                              EconomicBasis = row.Field<string>(nameof(IfrsVariable.EconomicBasis)),
                              Value = (-1.0) * doubleValue,
                                  };
}

In [ ]:
public class BenchmarkTestResult
{
    public string ErrorMessage {get; set;}
    public double ExpectedValue {get; set;}
    public double ComputedValue {get; set;}
    public BenchmarkTestResult (string message)
    {
        ErrorMessage = message;
    }
    public BenchmarkTestResult (string message, double expValue, double compValue)
    {
        ErrorMessage = message;
        ExpectedValue = expValue;
        ComputedValue = compValue;    
    }
}

In [ ]:
public List<BenchmarkTestResult> CompareAgainstBm (List<BenchmarkTestResult> errors, IEnumerable<IfrsVariable> expected, IEnumerable<IfrsVariable> computed)
{
    var expectedNotComputed = expected.Where(x => Math.Abs(x.Value) > BenchmarkPrecision).Select(x => x.ToIdentityString()).Except(computed.Select(x => x.ToIdentityString()));
    var computedNotExpected = computed.Where(x => Math.Abs(x.Value) > BenchmarkPrecision).Select(x => x.ToIdentityString()).Except(expected.Select(x => x.ToIdentityString()));
    if (expectedNotComputed.Any())
        foreach (var error in expectedNotComputed)
            errors.Add(new BenchmarkTestResult("Extra expected variable for: " + error));
    if (computedNotExpected.Any())
        foreach (var error in computedNotExpected)
            errors.Add(new BenchmarkTestResult("Extra computed variable for: " + error));
    
    var misMatchedVariables = expected.Concat(computed).AggregateOver().Where(x => Math.Abs(x.Value) > BenchmarkPrecision);
    if (misMatchedVariables.Any())
    {
        foreach (var variable in misMatchedVariables)
        {
          var message = variable.ToIdentityString();
          var bmValue = expected.FirstOrDefault(x => x.AocType == variable.AocType && 
                                                    x.Novelty == variable.Novelty &&
                                                    x.DataNode == variable.DataNode && 
                                                    x.EstimateType == variable.EstimateType && 
                                                    x.AmountType == variable.AmountType && 
                                                    x.EconomicBasis == variable.EconomicBasis && 
                                                    x.AccidentYear == variable.AccidentYear
                                                    )?.Value ?? 0.0d;
          var computedValue = computed.FirstOrDefault(x => x.AocType == variable.AocType && 
                                                    x.Novelty == variable.Novelty &&
                                                    x.DataNode == variable.DataNode && 
                                                    x.EstimateType == variable.EstimateType && 
                                                    x.AmountType == variable.AmountType && 
                                                    x.EconomicBasis == variable.EconomicBasis && 
                                                    x.AccidentYear == variable.AccidentYear
                                                     )?.Value ?? 0.0d;
          errors.Add(new BenchmarkTestResult("Value does not match for Variable: " + message, -bmValue, computedValue));
        }
    }
    return errors;
}

# Test Runner

In [ ]:
public bool ValidateMainTab(IDataRow mainRow, string reportingNode, int year, int month)
{
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Year)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainYear))
            throw new Exception("Value Year cannot be parsed."); 
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Month)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainMonth))
            throw new Exception("Value Month cannot be parsed."); 
    var mainReportingNode = mainRow[nameof(PartitionByReportingNodeAndPeriod.ReportingNode)].ToString();
    
    return reportingNode == mainReportingNode && year == mainYear && month == mainMonth;
}

In [ ]:
var errorList = new List<BenchmarkTestResult>();
foreach (var g in bmFiles.GroupBy(x => (ReportingNode: x.ReportingNode, Year: x.Year, Month: x.Month)))
{
    var reportingNode = g.Key.ReportingNode;
    var year = g.Key.Year;
    var month = g.Key.Month;
    string scenario = null;
    
    //Set up Args and storage
    await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = reportingNode, 
                                                                                Year= year, 
                                                                                Month = month, 
                                                                                Scenario = scenario});
    var partition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                        .SingleOrDefault(x => x.ReportingNode == reportingNode && x.Year== year && x.Month == month && x.Scenario == scenario)?.Id ?? new Guid();
    
    var computedVariablesByEstimateType = (await DataSource.Query<IfrsVariable>().ToArrayAsync()).ToDictionaryGrouped(x => x.EstimateType, x => x.ToArray());

    foreach (var bmfile in g)
    {
        var extension = System.IO.Path.GetExtension(bmfile.FileName);
        var stream = await Project.FileStorage.ReadAsync(bmfile.FileName);
        //Read Bm csv
        var bmDataTable = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

        var mainTab = bmDataTable.Tables[Main].Rows.First();
        if(!ValidateMainTab(mainTab, reportingNode, year, month))
            throw new Exception("Metadata and main tab of the Benchmark are different.");

        var bmTableName = bmDataTable.Tables.Single(x => x.TableName != Main).TableName;
        var bmVariables = bmDataTable.Tables[bmTableName].Rows.Select(x => CreateVariable(x, partition));

        //Get the computed Data
        var computedVariables = computedVariablesByEstimateType.TryGetValue(bmTableName, out var ret) ? ret : Enumerable.Empty<IfrsVariable>();

        if(!computedVariables.Any())    
            errorList.Add(new BenchmarkTestResult("No variables are computed for EstimateType: " + bmTableName, 0, 0));
        //CompareAgainstBm
        if (bmVariables.Any() && computedVariables.Any()) //TODO we are adding duplicates here
            errorList = errorList.Concat(CompareAgainstBm(errorList, bmVariables, computedVariables)).ToList();
    }
}
errorList = errorList.Distinct().ToList();

In [ ]:
errorList.Count().Should().Be(0);

In [ ]:
errorList